# Scraping Movies from Wikipedia

In [29]:
!pip install tmdbv3api


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [30]:
# Regular EDA (exploratory data analysis) and plotting libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import requests
import bs4 as bs
import urllib.request

from tmdbv3api import TMDb
import json
import requests

In [25]:
# Create dataframe to store all movies
df = pd.DataFrame()

# Scrape data from Wikipedia 
for year in [2018, 2019, 2020, 2021, 2022]:
    link = f'https://en.wikipedia.org/wiki/List_of_American_films_of_{year}'

    source = urllib.request.urlopen(link).read()
    soup = bs.BeautifulSoup(source,'lxml')
    
    tables = soup.find_all('table', class_='wikitable sortable')
    
    df1 = pd.read_html(str(tables[0]))[0]
    df2 = pd.read_html(str(tables[1]))[0]
    df3 = pd.read_html(str(tables[2]))[0]
    df4 = pd.read_html(str(tables[3]))[0]
    
    df = df.append(df1.append(df2.append(df3.append(df4, ignore_index=True), ignore_index=True), ignore_index=True), ignore_index=True)

In [26]:
df.head()

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,5.0,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2],NaN
1,JANUARY,5.0,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3],NaN
2,JANUARY,5.0,Stratton,Momentum Pictures / GFM Films,"Simon West (director); Duncan Falconer, Warren...",[4],NaN
3,JANUARY,10.0,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5],NaN
4,JANUARY,12.0,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6],NaN


In [27]:
df.shape

(1486, 7)

In [28]:
data = df[['Title','Cast and crew']]
data.head()

,Title,Cast and crew
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...
2,Stratton,"Simon West (director); Duncan Falconer, Warren..."
3,Sweet Country,"Warwick Thornton (director); David Tranter, St..."
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...


In [31]:
# Connect to TMDB API
tmdb = TMDb()
tmdb.api_key = 'your API key'

In [37]:
from tmdbv3api import Movie
tmdb_movie = Movie()

def get_genre(x): # Pass in the title of the movies
    genres = []
    result = tmdb_movie.search(x) # The title will be searched in the tmdb_movie
    if not result:
      return np.NaN
    else:
        movie_id = result[0].id # Match the "id" with the "title"
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id, tmdb.api_key)) # Get the result from the IMDb data
        data_json = response.json() # Convert it to a json file
        if data_json['genres']: # Extract the "genre"
            genre_str = ' ' 
            for i in range(0,len(data_json['genres'])):
                genres.append(data_json['genres'][i]['name']) # Add the "genre" to the empty genre list
            return genre_str.join(genres)
        else:
            np.NaN 
        
data['genres'] = data['Title'].map(lambda x: get_genre(str(x)))
data

/var/folders/zz/j2kt2p494g31x_wqy69bncdc0000gn/T/ipykernel_59769/288690785.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['genres'] = data['Title'].map(lambda x: get_genre(str(x)))


,Title,Cast and crew,genres
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Western Crime Thriller Drama
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller Mystery
...,...,...,...
1481,"Alice, Darling",Mary Nighy (director); Alanna Francis (screenp...,Thriller Mystery Drama
1482,NaN,NaN,Drama
1483,NaN,NaN,Drama
1484,NaN,NaN,Drama


In [38]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]
    
data['director_name'] = data['Cast and crew'].map(lambda x: get_director(str(x)))

/var/folders/zz/j2kt2p494g31x_wqy69bncdc0000gn/T/ipykernel_59769/3953848329.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['director_name'] = data['Cast and crew'].map(lambda x: get_director(str(x)))


In [39]:
def get_actor_1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

def get_actor_2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])
    
def get_actor_3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

data['actor_1_name'] = data['Cast and crew'].map(lambda x: get_actor_1(str(x)))
data['actor_2_name'] = data['Cast and crew'].map(lambda x: get_actor_2(str(x)))    
data['actor_3_name'] = data['Cast and crew'].map(lambda x: get_actor_3(str(x)))

/var/folders/zz/j2kt2p494g31x_wqy69bncdc0000gn/T/ipykernel_59769/3174978197.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['actor_1_name'] = data['Cast and crew'].map(lambda x: get_actor_1(str(x)))
/var/folders/zz/j2kt2p494g31x_wqy69bncdc0000gn/T/ipykernel_59769/3174978197.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['actor_2_name'] = data['Cast and crew'].map(lambda x: get_actor_2(str(x)))


In [40]:
data.head()

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Horror Mystery Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Western Crime Thriller Drama,Warwick Thornton,Bryan Brown,Sam Neill,NaN
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller Mystery,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson


In [41]:
# Rename the 'Title' column
data = data.rename(columns={'Title':'movie_title'})

In [42]:
new_data = data.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [43]:
new_data

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller,Insidious: The Last Key
1,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Thriller Drama,The Strange Ones
2,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Action Thriller,Stratton
3,Warwick Thornton,Bryan Brown,Sam Neill,NaN,Western Crime Thriller Drama,Sweet Country
4,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller Mystery,The Commuter
...,...,...,...,...,...,...
1481,Mary Nighy,Anna Kendrick,Kaniehtiio Horn,Charlie Carrick,Thriller Mystery Drama,"Alice, Darling"
1482,nan,nan,NaN,NaN,Drama,NaN
1483,nan,nan,NaN,NaN,Drama,NaN
1484,nan,nan,NaN,NaN,Drama,NaN


In [44]:
# Combine data for the tfidvectorizor
new_data['comb'] = new_data['actor_1_name'] + ' ' + new_data['actor_2_name'] + ' '+ new_data['actor_3_name'] + ' '+ new_data['director_name'] +' ' + new_data['genres']

In [45]:
# Check for missing values
new_data.isna().sum()

director_name      0
actor_1_name       0
actor_2_name      33
actor_3_name     133
genres             4
movie_title        5
comb             136
dtype: int64

In [46]:
# Drop missing values
new_data = new_data.dropna(how='any')

In [47]:
new_data['movie_title'] = new_data['movie_title'].str.lower()

/var/folders/zz/j2kt2p494g31x_wqy69bncdc0000gn/T/ipykernel_59769/2199470232.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['movie_title'] = new_data['movie_title'].str.lower()


In [48]:
new_data.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller,insidious: the last key,Lin Shaye Angus Sampson Leigh Whannell Adam Ro...
1,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Thriller Drama,the strange ones,Alex Pettyfer James Freedson-Jackson Emily Alt...
2,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Action Thriller,stratton,Dominic Cooper Austin Stowell Gemma Chan Simon...
4,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller Mystery,the commuter,Liam Neeson Vera Farmiga Patrick Wilson Jaume ...
5,Babak Najafi,Taraji P. Henson,Billy Brown,Danny Glover,Thriller Action Crime,proud mary,Taraji P. Henson Billy Brown Danny Glover Baba...


In [49]:
old_df = pd.read_csv('../data/data1.csv')
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,NaN,Documentary,star wars: episode vii - the force awakens ...,NaN
...,...,...,...,...,...,...,...
5359,Jim Strouse,Jessica Williams,Chris O'Dowd,Keith Stanfield,Romance Comedy,the incredible jessica james,Jessica Williams Chris O'Dowd Keith Stanfield ...
5360,Farhad Mann,Adelaide Kane,Benjamin Hollingsworth,Jean Louisa Kelly,Romance,can't buy my love,Adelaide Kane Benjamin Hollingsworth Jean Loui...
5361,Hannaleena Hauru,Inka Haapamäki,Rosa Honkonen,Tiitus Rantala,Romance Comedy,thick lashes of lauri mäntyvaara,Inka Haapamäki Rosa Honkonen Tiitus Rantala Ha...
5362,Jonathan A. Rosenbaum,Lou Diamond Phillips,Wallace Shawn,Gina Holden,Crime Comedy Action Family,cop and a half: new recruit,Lou Diamond Phillips Wallace Shawn Gina Holden...


In [51]:
final_df = old_df.append(new_data,ignore_index=True)
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,NaN,Documentary,star wars: episode vii - the force awakens ...,NaN
...,...,...,...,...,...,...,...
6709,Kasi Lemmons,Naomi Ackie,Stanley Tucci,Ashton Sanders,Music History Drama,whitney houston: i wanna dance with somebody,Naomi Ackie Stanley Tucci Ashton Sanders Kasi ...
6710,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,Crime,the pale blue eye,Christian Bale Harry Melling Gillian Anderson ...
6711,Sarah Polley,Rooney Mara,Claire Foy,Jessie Buckley,Drama,women talking,Rooney Mara Claire Foy Jessie Buckley Sarah Po...
6712,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,Comedy Drama,a man called otto,Tom Hanks Mariana Treviño Rachel Keller Marc F...


In [53]:
# Save data into a csv file
final_df.to_csv('../data/preprocessed_data.csv', index=False)